### A friend of mine wants to move to a specific town in Hartford County, Connecticut.  She's interested in what people do for fun/entertainment in the vicinity.  However, she doesn't know which other towns are similar.   

### This project will investigate the towns in Hartford County and find similarities between them using Foursquare to look at the trending venues.  The identified towns will then be segmented and clustsered.

### Let's start by reading in data from the web.

In [83]:
#import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [84]:
#read html of table from website into "data"; put desired data into a dataframe, dropping extraneous columns
CT_data = pd.read_html('https://en.wikipedia.org/wiki/List_of_towns_in_Connecticut')
CT_table = CT_data[1]
CT_df = pd.DataFrame(CT_table)
CT_df = CT_df[['County','Town']]
CT_df.head(15)

,County,Town
0,Tolland County,Andover
1,New Haven County,Ansonia
2,Windham County,Ashford
3,Hartford County,Avon
4,Litchfield County,Barkhamsted
5,New Haven County,Beacon Falls
6,Hartford County,Berlin
7,New Haven County,Bethany
8,Fairfield County,Bethel
9,Litchfield County,Bethlehem


In [85]:
#sort by County
CT_df = CT_df.sort_values(by = "County", ascending = True)
CT_df.head(15)

,County,Town
84,Fairfield County,Monroe
34,Fairfield County,Darien
46,Fairfield County,Easton
157,Fairfield County,Westport
50,Fairfield County,Fairfield
56,Fairfield County,Greenwich
156,Fairfield County,Weston
89,Fairfield County,New Canaan
33,Fairfield County,Danbury
90,Fairfield County,New Fairfield


In [86]:
#reset index so numbering is sequential, starting at zero
CT_df.reset_index(inplace = True, drop = True)
CT_df.head(50)

,County,Town
0,Fairfield County,Monroe
1,Fairfield County,Darien
2,Fairfield County,Easton
3,Fairfield County,Westport
4,Fairfield County,Fairfield
5,Fairfield County,Greenwich
6,Fairfield County,Weston
7,Fairfield County,New Canaan
8,Fairfield County,Danbury
9,Fairfield County,New Fairfield


In [87]:
CT_df.shape

(170, 2)

### Now, let's add latitude and longitude coordinates to the dataframe.

In [88]:
#import libraries to complete further analysis
import numpy as np #library to handle data in a vectorized manner

import pandas as pd #library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json #library to handle JSON files
#!conda update -n base -c defaults conda
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim #converts an address into latitude and longitude values

import requests #library to handle requests
from pandas.io.json import json_normalize #tranforms JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium #map rendering library

print('Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [89]:
#create empty lists to receive data

lat_col = []   
long_col = []
geolocator = Nominatim(user_agent="ct_explorer")

# iterate over rows in dataframe
for index, row in CT_df.iterrows(): 


    City = row['Town']
    State = 'Connecticut'       
    address = str(City)+','+str(State)

    results = geolocator.geocode(address)   
    lat = results.latitude
    long = results.longitude

    lat_col.append(lat)
    long_col.append(long)

# Define "latitude" and "Longitude" columns from list created above    

CT_df['Latitude'] = lat_col

CT_df['Longitude'] = long_col

In [90]:
CT_df.head(50)

,County,Town,Latitude,Longitude
0,Fairfield County,Monroe,41.332596,-73.207336
1,Fairfield County,Darien,41.078708,-73.469287
2,Fairfield County,Easton,41.252874,-73.297339
3,Fairfield County,Westport,41.141486,-73.357895
4,Fairfield County,Fairfield,41.141208,-73.263726
5,Fairfield County,Greenwich,41.026486,-73.628460
6,Fairfield County,Weston,41.202130,-73.381274
7,Fairfield County,New Canaan,41.146763,-73.494845
8,Fairfield County,Danbury,41.394817,-73.454011
9,Fairfield County,New Fairfield,41.466483,-73.485679


### Next, let's make a map of Connecticut using the latitude and longitude values.

In [91]:
# create map of Connecticut using latitude and longitude values

map_ct = folium.Map(location=[lat, long], zoom_start=10)

# add markers to map
for lat, lng, county, town in zip(CT_df['Latitude'], CT_df['Longitude'], CT_df['County'], CT_df['Town']):
    label = '{}, {}'.format(town, county)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ct)  

map_ct

### Now that a map of Connecticut has been created, let's focus on the specific county that my friend is interested in: Hartford (creating the map above may help guide future exploration, if necessary, another county adjacent to Hartford could also be investigated or included .

### For now, let's focus on Hartford County.

In [92]:
hart_df = CT_df[CT_df['County'] == 'Hartford County'].reset_index(drop=True)
hart_df.head(30)

,County,Town,Latitude,Longitude
0,Hartford County,East Hartford,41.782322,-72.612035
1,Hartford County,Rocky Hill,41.664822,-72.639259
2,Hartford County,East Windsor,41.915632,-72.613073
3,Hartford County,Wethersfield,41.714267,-72.652592
4,Hartford County,Newington,41.697878,-72.723706
5,Hartford County,Enfield,41.978939,-72.575511
6,Hartford County,Farmington,41.719822,-72.832043
7,Hartford County,Glastonbury,41.712322,-72.608146
8,Hartford County,Granby,41.953986,-72.788707
9,Hartford County,Plainville,41.671140,-72.867243


In [93]:
hart_df.shape

(29, 4)

In [94]:
# Get the coordinates of Hartford County,CT
address = 'Hartford County, Connecticut'

geolocator = Nominatim(user_agent="ct_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Hartford County are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Hartford County are 41.7923478, -72.722324.


#### Map of Hartford County, CT and its towns.

In [95]:
# create map of Hartford County using latitude and longitude values
map_hart = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(hart_df['Latitude'], hart_df['Longitude'], hart_df['Town']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hart)  
    
map_hart

### Now, it's time to use Foursquare to find the popular places of interest.  

### We will start by examining the town to which my friend is interested in moving, Avon.  Then, other towns will be explored, segmented, and clustered.  This should give my friend a good idea of towns similar to the target town and what distinguishes that cluster of towns.

In [96]:
# The code was removed by Watson Studio for sharing.

In [97]:

hart_df.loc[27, 'Town']

'Avon'

In [98]:
town_latitude = hart_df.loc[27, 'Latitude'] # town latitude value
town_longitude = hart_df.loc[27, 'Longitude'] # town longitude value

town_name = hart_df.loc[27, 'Town'] # town name

print('Latitude and longitude values of {} are {}, {}.'.format(town_name, 
                                                               town_latitude, 
                                                               town_longitude))

Latitude and longitude values of Avon are 41.8098209, -72.8306541.


Now, let's get the top 100 venues that are within a radius of 500 meters of Avon. 

First, let's create the GET request URL.

In [99]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    town_latitude, 
    town_longitude, 
    radius, 
    LIMIT)


In [100]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed18fbdc8cff2001bdb12a7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Avon',
  'headerFullLocation': 'Avon',
  'headerLocationGranularity': 'city',
  'totalResults': 16,
  'suggestedBounds': {'ne': {'lat': 41.8143209045, 'lng': -72.82462802939108},
   'sw': {'lat': 41.805320895499996, 'lng': -72.83668017060893}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '554a620c498ec595b67cbe32',
       'name': "Dom's Coffee",
       'location': {'lat': 41.810382422557076,
        'lng': -72.83232758345521,
        'labeledLatLngs': [{'label': 'display',
          'lat': 41.810382422557076,
          'lng': -7

In [101]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [102]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(16)

,name,categories,lat,lng
0,Dom's Coffee,Coffee Shop,41.810382,-72.832328
1,Elephant Trail,Thai Restaurant,41.808008,-72.828137
2,First & Last Tavern,Italian Restaurant,41.810954,-72.832292
3,Da Capo,Italian Restaurant,41.809174,-72.829948
4,The UPS Store,Shipping Store,41.807955,-72.827703
5,SUBWAY,Sandwich Place,41.808727,-72.828960
6,Bruegger's Bagel Bakery,Bagel Shop,41.807357,-72.827398
7,The Coffee Trade Inc.,Coffee Shop,41.809926,-72.832480
8,Residence Inn Hartford Avon,Hotel,41.812420,-72.831833
9,Pick and Mix,Korean Restaurant,41.807818,-72.827272


In [103]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


### Now that Avon has been explored, let's go ahead and look at the other towns in Hartford County.

In [104]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Following is the code to run the above function on each town and create a new dataframe called hartford_venues:

In [105]:
hartford_venues = getNearbyVenues(names=hart_df['Town'],
                                   latitudes=hart_df['Latitude'],
                                   longitudes=hart_df['Longitude']
                                  )

East Hartford
Rocky Hill
East Windsor
Wethersfield
Newington
Enfield
Farmington
Glastonbury
Granby
Plainville
Hartland
Hartford
Berlin
East Granby
Burlington
South Windsor
Windsor
Canton
Marlborough
West Hartford
Suffield
Southington
New Britain
Manchester
Bloomfield
Simsbury
Bristol
Avon
Windsor Locks


In [106]:
print(hartford_venues.shape)
hartford_venues.head()

(383, 7)


,Town,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,East Hartford,41.782322,-72.612035,Dollar Tree,41.779490,-72.610046,Discount Store
1,East Hartford,41.782322,-72.612035,Shell,41.783267,-72.613054,Gas Station
2,East Hartford,41.782322,-72.612035,Rite Aid,41.779700,-72.610099,Pharmacy
3,East Hartford,41.782322,-72.612035,Family Dollar,41.778808,-72.612192,Discount Store
4,East Hartford,41.782322,-72.612035,Granny's Pie Factory,41.782634,-72.612840,Dessert Shop


In [107]:
hartford_venues.groupby('Town').count()

,Town Latitude,Town Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Town,,,,,,
Avon,16,16,16,16,16,16
Berlin,5,5,5,5,5,5
Bloomfield,9,9,9,9,9,9
Bristol,14,14,14,14,14,14
Burlington,11,11,11,11,11,11
Canton,11,11,11,11,11,11
East Granby,7,7,7,7,7,7
East Hartford,14,14,14,14,14,14
East Windsor,11,11,11,11,11,11


In [108]:
print('There are {} unique categories.'.format(len(hartford_venues['Venue Category'].unique())))

There are 117 unique categories.


## Analysis of each town in Hartford County

We need to use one hot encoding so that the categorical variables can be processed.

In [109]:
# one hot encoding
hartford_onehot = pd.get_dummies(hartford_venues[['Venue Category']], prefix="", prefix_sep="")

# addtown column back to dataframe
hartford_onehot['Town'] = hartford_venues['Town'] 

# move neighborhood column to the first column
fixed_columns = [hartford_onehot.columns[-1]] + list(hartford_onehot.columns[:-1])
hartford_onehot = hartford_onehot[fixed_columns]

hartford_onehot.head()

,Town,ATM,Alternative Healer,American Restaurant,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Business Service,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Elementary School,Event Service,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Lawyer,Liquor Store,Lounge,Market,Massage Studio,Mexican Restaurant,Museum,Nail Salon,New American Restaurant,Noodle House,Outlet Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Wine Shop,Wings Joint,Yoga Studio
0,East Hartford,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,East Hartford,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,East Hartford,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,East Hartford,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,East Hartford,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [110]:
hartford_onehot.shape

(383, 118)

In [111]:
hartford_grouped = hartford_onehot.groupby('Town').mean().reset_index()
hartford_grouped

,Town,ATM,Alternative Healer,American Restaurant,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bike Shop,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Station,Business Service,Café,Campground,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Elementary School,Event Service,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food,Food Court,Food Truck,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hobby Shop,Home Service,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Latin American Restaurant,Lawyer,Liquor Store,Lounge,Market,Massage Studio,Mexican Restaurant,Museum,Nail Salon,New American Restaurant,Noodle House,Outlet Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Spa,Speakeasy,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Thai Restaurant,Thrift / Vintage Store,Trail,Train Station,Wine Shop,Wings Joint,Yoga Studio
0,Avon,0.0,0.0000,0.000000,0.000000,0.0625,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0000,0.000000,0.000000,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.062500,0.0625,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.0625,0.062500,0.00,0.000000,0.000000,0.0000,0.062500,0.0,0.000000,0.000000,0.062500,0.000000,0.000000,0.0000,0.0625,0.00,0.000000
1,Berlin,0.0,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.2,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.00,0.000000
2,Bloomfield,0.0,0.0000,0.111111,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.222222,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,

In [112]:
hartford_grouped.shape

(28, 118)

In [113]:
num_top_venues = 5

for town in hartford_grouped['Town']:
    print("----"+town+"----")
    temp = hartford_grouped[hartford_grouped['Town'] == town].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Avon----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.12
2     Thai Restaurant  0.06
3      Sandwich Place  0.06
4  Seafood Restaurant  0.06


----Berlin----
            venue  freq
0  Breakfast Spot   0.2
1     Pizza Place   0.2
2      Donut Shop   0.2
3     Flower Shop   0.2
4       Bike Shop   0.2


----Bloomfield----
                 venue  freq
0                 Bank  0.22
1           Donut Shop  0.11
2  American Restaurant  0.11
3    Convenience Store  0.11
4         Liquor Store  0.11


----Bristol----
            venue  freq
0     Pizza Place  0.14
1   Grocery Store  0.07
2      Food Truck  0.07
3    Dessert Shop  0.07
4  Discount Store  0.07


----Burlington----
                        venue  freq
0  Construction & Landscaping  0.09
1                       Plaza  0.09
2           Convenience Store  0.09
3            Asian Restaurant  0.09
4              Farmers Market  0.09


----Canton----
                    venue  freq
0             C

Let's put that into a pandas dataframe.

First, let's write a function to sort the venues in descending order.


In [114]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each town.

In [115]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
towns_venues_sorted = pd.DataFrame(columns=columns)
towns_venues_sorted['Town'] = hartford_grouped['Town']

for ind in np.arange(hartford_grouped.shape[0]):
    towns_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hartford_grouped.iloc[ind, :], num_top_venues)

towns_venues_sorted.head()

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Avon,Coffee Shop,Italian Restaurant,Furniture / Home Store,Juice Bar,Sandwich Place,Seafood Restaurant,Shipping Store,Sporting Goods Shop,Bagel Shop,Athletics & Sports
1,Berlin,Bike Shop,Pizza Place,Flower Shop,Donut Shop,Breakfast Spot,Event Service,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio
2,Bloomfield,Bank,Convenience Store,Liquor Store,Restaurant,Pharmacy,Donut Shop,Caribbean Restaurant,American Restaurant,Dance Studio,Elementary School
3,Bristol,Pizza Place,Sandwich Place,Grocery Store,Brewery,Pharmacy,Dessert Shop,Bank,Discount Store,Donut Shop,Thai Restaurant
4,Burlington,Construction & Landscaping,Flower Shop,Irish Pub,Plaza,Pizza Place,Pharmacy,Farmers Market,Italian Restaurant,Campground,Asian Restaurant


## Cluster the Towns in Hartford County

In [116]:
# set number of clusters
kclusters = 4

hartford_grouped_clustering = hartford_grouped.drop('Town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hartford_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:28] 

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 2, 1, 1, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each town.

In [117]:
# add clustering labels
towns_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hartford_merged = hart_df

# merge hartford_grouped with hart_df to add latitude/longitude for each town
hartford_merged = hartford_merged.join(towns_venues_sorted.set_index('Town'), on='Town')

In [118]:
#hartford_merged.drop('Cluster labels', axis = 1)
hartford_merged.head(10) # check the last columns!

,County,Town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hartford County,East Hartford,41.782322,-72.612035,1.0,Discount Store,Liquor Store,Shipping Store,Dessert Shop,Pizza Place,Home Service,Pharmacy,Chinese Restaurant,Automotive Shop,Caribbean Restaurant
1,Hartford County,Rocky Hill,41.664822,-72.639259,1.0,ATM,Steakhouse,Liquor Store,Business Service,Dance Studio,Pharmacy,Deli / Bodega,Bridal Shop,Sandwich Place,Donut Shop
2,Hartford County,East Windsor,41.915632,-72.613073,1.0,Fast Food Restaurant,Hotel,Fried Chicken Joint,Asian Restaurant,Diner,Sushi Restaurant,Donut Shop,Sporting Goods Shop,Pharmacy,Yoga Studio
3,Hartford County,Wethersfield,41.714267,-72.652592,1.0,Farm,Yoga Studio,Bar,Deli / Bodega,Pizza Place,Event Space,Museum,Big Box Store,Farmers Market,Ice Cream Shop
4,Hartford County,Newington,41.697878,-72.723706,1.0,American Restaurant,Pharmacy,Hobby Shop,Bank,Bar,Chinese Restaurant,Shoe Store,Market,Nail Salon,Burger Joint
5,Hartford County,Enfield,41.978939,-72.575511,3.0,Chinese Restaurant,Garden Center,Food,Flower Shop,Yoga Studio,Elementary School,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio
6,Hartford County,Farmington,41.719822,-72.832043,1.0,Elementary School,Food,Home Service,Lawyer,Yoga Studio,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega
7,Hartford County,Glastonbury,41.712322,-72.608146,1.0,Italian Restaurant,Pizza Place,Bakery,Ice Cream Shop,Salon / Barbershop,Coffee Shop,Thai Restaurant,Grocery Store,Cocktail Bar,Jewelry Store
8,Hartford County,Granby,41.953986,-72.788707,1.0,Ice Cream Shop,Grocery Store,Chinese Restaurant,Sandwich Place,Liquor Store,Coffee Shop,Mexican Restaurant,Dessert Shop,Italian Restaurant,Park
9,Hartford County,Plainville,41.671140,-72.867243,3.0,Mexican Restaurant,Convenience Store,Pizza Place,Bakery,Burger Joint,Diner,Yoga Studio,Elementary School,Cosmetics Shop,Credit Union


In [119]:
hartford_merged.shape

(29, 15)

In [120]:
hartford_merged.tail(29)

,County,Town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hartford County,East Hartford,41.782322,-72.612035,1.0,Discount Store,Liquor Store,Shipping Store,Dessert Shop,Pizza Place,Home Service,Pharmacy,Chinese Restaurant,Automotive Shop,Caribbean Restaurant
1,Hartford County,Rocky Hill,41.664822,-72.639259,1.0,ATM,Steakhouse,Liquor Store,Business Service,Dance Studio,Pharmacy,Deli / Bodega,Bridal Shop,Sandwich Place,Donut Shop
2,Hartford County,East Windsor,41.915632,-72.613073,1.0,Fast Food Restaurant,Hotel,Fried Chicken Joint,Asian Restaurant,Diner,Sushi Restaurant,Donut Shop,Sporting Goods Shop,Pharmacy,Yoga Studio
3,Hartford County,Wethersfield,41.714267,-72.652592,1.0,Farm,Yoga Studio,Bar,Deli / Bodega,Pizza Place,Event Space,Museum,Big Box Store,Farmers Market,Ice Cream Shop
4,Hartford County,Newington,41.697878,-72.723706,1.0,American Restaurant,Pharmacy,Hobby Shop,Bank,Bar,Chinese Restaurant,Shoe Store,Market,Nail Salon,Burger Joint
5,Hartford County,Enfield,41.978939,-72.575511,3.0,Chinese Restaurant,Garden Center,Food,Flower Shop,Yoga Studio,Elementary School,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio
6,Hartford County,Farmington,41.719822,-72.832043,1.0,Elementary School,Food,Home Service,Lawyer,Yoga Studio,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega
7,Hartford County,Glastonbury,41.712322,-72.608146,1.0,Italian Restaurant,Pizza Place,Bakery,Ice Cream Shop,Salon / Barbershop,Coffee Shop,Thai Restaurant,Grocery Store,Cocktail Bar,Jewelry Store
8,Hartford County,Granby,41.953986,-72.788707,1.0,Ice Cream Shop,Grocery Store,Chinese Restaurant,Sandwich Place,Liquor Store,Coffee Shop,Mexican Restaurant,Dessert Shop,Italian Restaurant,Park
9,Hartford County,Plainville,41.671140,-72.867243,3.0,Mexican Restaurant,Convenience Store,Pizza Place,Bakery,Burger Joint,Diner,Yoga Studio,Elementary School,Cosmetics Shop,Credit Union


In [121]:
#drop row with NaN
hartford_merged = hartford_merged.drop(hartford_merged.index[10])
hartford_merged

,County,Town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hartford County,East Hartford,41.782322,-72.612035,1.0,Discount Store,Liquor Store,Shipping Store,Dessert Shop,Pizza Place,Home Service,Pharmacy,Chinese Restaurant,Automotive Shop,Caribbean Restaurant
1,Hartford County,Rocky Hill,41.664822,-72.639259,1.0,ATM,Steakhouse,Liquor Store,Business Service,Dance Studio,Pharmacy,Deli / Bodega,Bridal Shop,Sandwich Place,Donut Shop
2,Hartford County,East Windsor,41.915632,-72.613073,1.0,Fast Food Restaurant,Hotel,Fried Chicken Joint,Asian Restaurant,Diner,Sushi Restaurant,Donut Shop,Sporting Goods Shop,Pharmacy,Yoga Studio
3,Hartford County,Wethersfield,41.714267,-72.652592,1.0,Farm,Yoga Studio,Bar,Deli / Bodega,Pizza Place,Event Space,Museum,Big Box Store,Farmers Market,Ice Cream Shop
4,Hartford County,Newington,41.697878,-72.723706,1.0,American Restaurant,Pharmacy,Hobby Shop,Bank,Bar,Chinese Restaurant,Shoe Store,Market,Nail Salon,Burger Joint
5,Hartford County,Enfield,41.978939,-72.575511,3.0,Chinese Restaurant,Garden Center,Food,Flower Shop,Yoga Studio,Elementary School,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio
6,Hartford County,Farmington,41.719822,-72.832043,1.0,Elementary School,Food,Home Service,Lawyer,Yoga Studio,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega
7,Hartford County,Glastonbury,41.712322,-72.608146,1.0,Italian Restaurant,Pizza Place,Bakery,Ice Cream Shop,Salon / Barbershop,Coffee Shop,Thai Restaurant,Grocery Store,Cocktail Bar,Jewelry Store
8,Hartford County,Granby,41.953986,-72.788707,1.0,Ice Cream Shop,Grocery Store,Chinese Restaurant,Sandwich Place,Liquor Store,Coffee Shop,Mexican Restaurant,Dessert Shop,Italian Restaurant,Park
9,Hartford County,Plainville,41.671140,-72.867243,3.0,Mexican Restaurant,Convenience Store,Pizza Place,Bakery,Burger Joint,Diner,Yoga Studio,Elementary School,Cosmetics Shop,Credit Union


Finally, let's visualize the resulting clusters!

In [122]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hartford_merged['Latitude'], hartford_merged['Longitude'], hartford_merged['Town'], hartford_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

Now, each cluster can be examined to determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, a name can be assigned to each cluster.

### Cluster 1

In [123]:
hartford_merged.loc[hartford_merged['Cluster Labels'] == 0, hartford_merged.columns[[1] + list(range(5, hartford_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Windsor Locks,American Restaurant,Deli / Bodega,Yoga Studio,Event Space,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Dessert Shop,Diner


### Cluster 2

In [124]:
hartford_merged.loc[hartford_merged['Cluster Labels'] == 1, hartford_merged.columns[[1] + list(range(5, hartford_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Hartford,Discount Store,Liquor Store,Shipping Store,Dessert Shop,Pizza Place,Home Service,Pharmacy,Chinese Restaurant,Automotive Shop,Caribbean Restaurant
1,Rocky Hill,ATM,Steakhouse,Liquor Store,Business Service,Dance Studio,Pharmacy,Deli / Bodega,Bridal Shop,Sandwich Place,Donut Shop
2,East Windsor,Fast Food Restaurant,Hotel,Fried Chicken Joint,Asian Restaurant,Diner,Sushi Restaurant,Donut Shop,Sporting Goods Shop,Pharmacy,Yoga Studio
3,Wethersfield,Farm,Yoga Studio,Bar,Deli / Bodega,Pizza Place,Event Space,Museum,Big Box Store,Farmers Market,Ice Cream Shop
4,Newington,American Restaurant,Pharmacy,Hobby Shop,Bank,Bar,Chinese Restaurant,Shoe Store,Market,Nail Salon,Burger Joint
6,Farmington,Elementary School,Food,Home Service,Lawyer,Yoga Studio,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega
7,Glastonbury,Italian Restaurant,Pizza Place,Bakery,Ice Cream Shop,Salon / Barbershop,Coffee Shop,Thai Restaurant,Grocery Store,Cocktail Bar,Jewelry Store
8,Granby,Ice Cream Shop,Grocery Store,Chinese Restaurant,Sandwich Place,Liquor Store,Coffee Shop,Mexican Restaurant,Dessert Shop,Italian Restaurant,Park
11,Hartford,Coffee Shop,Café,Bar,Bakery,Gas Station,Bus Station,Discount Store,Donut Shop,Speakeasy,New American Restaurant
12,Berlin,Bike Shop,Pizza Place,Flower Shop,Donut Shop,Breakfast Spot,Event Service,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio


### Cluster 3

In [125]:
hartford_merged.loc[hartford_merged['Cluster Labels'] == 2, hartford_merged.columns[[1] + list(range(5, hartford_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,West Hartford,Clothing Store,Event Service,Construction & Landscaping,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio,Deli / Bodega,Dessert Shop,Diner


### Cluster 4

In [128]:
hartford_merged.loc[hartford_merged['Cluster Labels'] == 3, hartford_merged.columns[[1] + list(range(5, hartford_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Enfield,Chinese Restaurant,Garden Center,Food,Flower Shop,Yoga Studio,Elementary School,Convenience Store,Cosmetics Shop,Credit Union,Dance Studio
9,Plainville,Mexican Restaurant,Convenience Store,Pizza Place,Bakery,Burger Joint,Diner,Yoga Studio,Elementary School,Cosmetics Shop,Credit Union
14,Burlington,Construction & Landscaping,Flower Shop,Irish Pub,Plaza,Pizza Place,Pharmacy,Farmers Market,Italian Restaurant,Campground,Asian Restaurant
